In [ ]:
import itertools as its
import math as m 


def CreateBlockMatrix(dimensions, blockshape, blocklist):
    '''Create one matrix out of several specified blocks'''
    return matrix.block(dimensions[0],dimensions[1], [blocklist[t] for t in blockshape])


def switchingmatrixinf(level, blocknumber, sort):
    '''Create a Q(level,blocknumber,sort)'''
    J = ones_matrix(level)
    O = zero_matrix(level)
    Y = level*identity_matrix(level)- J

    def rotate(l):
        return [l[-1]] + l[:-1]

    baserow = [0,2] + [1]*(blocknumber-2)
    shapematrix = []
    for _ in range(blocknumber):
        shapematrix = shapematrix + baserow
        baserow = rotate(baserow)
        
    return CreateBlockMatrix([blocknumber,blocknumber], shapematrix, [sort*Y,O,J])/level

def Qresp(Q,m):
    '''Find all Q-respecting vectors for any Q of size m'''
    qrespvectors = []
    for vec in its.product(range(2), repeat = m):
        if all(i in {0,1} for i in vector(vec)*Q):
            qrespvectors.append(vec)
    
    return qrespvectors
    
def BuildGraphsWithSwitching(outersize, switchingsize, switchinggraphs, qrespvec, symmetry_column):
    '''Given a list of switching graphs and a set of Q-respecting vectors construct all matrices as in T_n. 
    Symmetries of Q can be incorporated by only selecting a subset of the Q-respecting vectors as options for the symmetry_column.'''
    allgraphswithswitching = []
    if outersize > 0:
        allvalid_edges = [matrix(outersize-1, switchingsize, M) for M in its.product(qrespvec, repeat=outersize-1)]
    else:
        return switchinggraphs
    
    for A in switchinggraphs:
        for W in symmetry_column:
            for V in allvalid_edges:
                V = V.stack(matrix(1, switchingsize, W))
                switchingmatrix = A.augment(V.transpose())
                for C in graphs(outersize):
                    bottommatrix= V.augment(C.adjacency_matrix())
                    allgraphswithswitching.append(switchingmatrix.stack(bottommatrix))
        
    return allgraphswithswitching


def checkcospectralmate(Q, graphswithswitching):
    '''Check for each graph if it has a cospectral mate through Q-switching. 
    If so, add the graph, the switched graphs and their complements to the list. 
    At the end reduce the list to a set which only contains non-isomorphic graphs.'''

    #Here we use that B is also a graph with a cospectral mate through Q-switching. Is that true?
    hascospectralmate = []
    for A in graphswithswitching:
        Bcanlabel =  Graph(Q.transpose()*A*Q).canonical_label()
        Acanlabel = Graph(A).canonical_label()
        if Bcanlabel != Acanlabel:
            G = Acanlabel.complement()
            H = Bcanlabel.complement()
            hascospectralmate += [Acanlabel, Bcanlabel, G.canonical_label(), H.canonical_label()]
    
    immutablegraphs = [G.copy(immutable=True) for G in hascospectralmate]
    return set(immutablegraphs)



For GM and WQH

In [ ]:

def orbitQinf(n, blocks, level, sort, recursionlevel):
    '''This function recursively finds all permutations (as matrices) that do not fix the switching matrix Q of the infinite family.
    At each recursion level it takes any set of size equal to the level (containing numbers that we not put to the front before) and puts it at the front of the permutation. 
    The matrix Q has a symmetry group induced by rotating the blocks and any permutation within the blocks. 
    Therefore it only matters what the set of vertices in each block are, so we pick combinations 
    and by picking from [1,....,blocks*level-1] we ensure that the last block contains the highest number, so we deal with the symmetry of the block rotation.'''

    #I could remove n and replace by blocks*level just have to add identity matrix to P1
    if blocks == 2 and sort == 'gm':
        return [identity_matrix(n)]
    if recursionlevel == 1:
        return [identity_matrix(n)]

    def getvertices(n, S):
        #pulls vertices in S to the front of the list.
        complement = set(range(1,n+1)) - set(S)
        return S + list(complement)

    matrices= []
    for X in Combinations(range(level*(blocks-recursionlevel)+1, blocks*level), level):
        Xmatrix = Permutation(getvertices(n, X)).to_matrix()
        for P in symmetrymatrices(n, blocks, level, sort, recursionlevel-1):
            # As the matrices will be transposed (on the left) in bruteswitching. 
            # The way to ensure that the highest level does the second to last block is to multiply Xmatrix on the left. 
            matrices.append(Xmatrix*P)

    return matrices

def Genswitchingset(level, blocks, sort):
    '''Create all Gamma that are a switching graph (up to complementation) for a Q from the infinite family
    by simply going through all graphs up to half the vertices and testing whether they are a switching graph for any permutation of Q.'''
    switchingset = []

    Q = switchingmatrixinf(level, blocks, sort)
    Permlist = orbitQinf(blocks*level, blocks, level, sort, blocks)
    n = level*blocks
    for G in graphs.nauty_geng(str(n) + ' 0:' + str(m.floor(n*(n-1)/4))):
        A = G.adjacency_matrix()
        for P in Permlist:
            Aprime = Q.transpose()*P.transpose()*A*P*Q
            if all(all(entry in {0,1} for entry in row) for row in Aprime.rows()):
                switchingset.append(P.transpose()*A*P)

    return [k for k, _ in its.groupby(sorted(switchingset))]


################################################
def BuildandEnumerateCospectralgraphs(level,  sort, added_vertices):
    '''This function combines the earlier methods to build all graphs which have a cospectral mate
    via a WQH or GM switching method.'''
    Q = switchingmatrixinf(level, 2, sort)
    qrespvectors = Qresp(Q, 2*level)
    switchingsets = Genswitchingset(level,  sort)


    #To ensure not making too many graphs in T_n we do not use Q-respecting vectors in the symmetry column
    # that also appear as the image of an earlier Q-respecting vectors.    
    qresp_with_symmetry = []
    temp_qresp = []
    Q = switchingmatrixinf(level, sort)
    
    for x in goodedges:
        y = vector(x)
        if y not in temp_qresp:
            temp_qresp.append(y)
            temp_qresp.append(y*Q)
            qresp_with_symmetry.append(y)

    HasQswitching = BuildGraphsWithSwitching(added_vertices, 2*level, switchingsets, qrespvectors, qresp_with_symmetry)
  
    Q = switchingmatrixinf(level, 2, sort)
    Q = Q.block_sum(identity_matrix(added_vertices))

    return checkcospectralmate(Q, HasQswitching)


AH6 and Fano switching. These switching methods have specific switching graphs that correspond to irreducible switching methods. These will be added by hand as switching graphs.

In [ ]:
def BuildandEnumerateGeneral(Q, added_vertices, switchingsets, symmetry_column = None):
    '''Create all graphs that have a cospectral mate through (Q,Gamma)-switching where Gamma in switchingsets.'''
    m = Q.dimensions()[0]
    goodedges = Qresp(Q, m)
    if symmetry_column is None:
       symmetry_column = goodedges

    HasQswitching = BuildGraphsWithSwitching(added_vertices, m, switchingsets, goodedges, symmetry_column)
    Q = Q.block_sum(identity_matrix(added_vertices))

    return checkcospectralmate(Q, HasQswitching)

#####################################################

#For AH switching these are the vectors for the symmetry_column 
qvectors_uptosymmetry = [vector([0,0,0,0,0,0]), vector([1,1,1,1,1,1]), vector([1,1,0,0,0,0]), vector([1,1,1,1,0,0]), vector([1,0,1,0,1,0]), vector([0,1,0,1,0,1]), vector([0,1,1,0,1,0]), vector([0,1,0,1,1,0])]

#This is the irreducible matrix
O = zero_matrix(2)
N = matrix([[0,0],[1,1]])
irredmatrixAH = CreateBlockMatrix((3,3), [0,1,2,2,0,1,1,2,0], [O, N, N.transpose()])

Q_AH = switchingmatrixinf(2,3,-1)

###############################################

#The orthogonal matrix for Fano switching
def rotate(l):
    return [l[-1]] + l[:-1]

baserow = [-1,1,1,0,1,0,0]
switchingmat = []
for _ in range(7):
        switchingmat.append(baserow)
        baserow = rotate(baserow)
Q_fano = 1/2*matrix(switchingmat)

#For Fano switching these are the irreducible matrices.
special = matrix([[0,0,0,0,0,1,1], [0,0,0,0,0,0,1], [0,0,0,1,0,1,1], [0,0,0,0,1,0,1], [0,0,0,0,0,1,0], [0,0,0,0,0,0,1], [0,0,0,0,0,0,0]])
special = special + special.T
irredmatrixfano = [graphs.CycleGraph(7).adjacency_matrix(), special]

################################################


In [ ]:
def export(filename, object_list, method_name):
    '''Function to save list to a file'''
    with open(filename, 'w') as f:
        for obj in object_list:
            f.write(attrcall(method_name)(obj) + '\n')

##################################################

#Find all cospectral mates through BuildandEnumerateCospectralgraphs or BuildandEnumerateGeneral
#see here an example
Graphswithcospectralmates = BuildandEnumerateGeneral(Q_fano, 2, irredmatrixfano, 1)

#Then save the list of graph6 strings to a file
export('enumeration.g6', Graphswithcospectralmates, 'graph6_string')